In [ ]:
# PROJECT - Retail Orders Sales 

#### Generate a JSON Format token connect public API to download the data set

In [1]:
import pandas as pd
import kaggle
import openpyxl

1. Generate a JSON format token from kaggle website. We find it in the Downloads File.
2. Copy from Download Files & Paste it in 'Home Directory' > Amir > kaggle.
3. To download the data set, run the following command: 
!kaggle datasets download ankitbansa106/retail-orders -f orders.csv

In [3]:
!kaggle datasets download ankitbansa106/retail-orders -f orders.csv

'kaggle' is not recognized as an internal or external command,
operable program or batch file.


In [4]:
import os
print(os.getcwd())  # to see your working directory


c:\Users\Amir\Dropbox\Workshop demo\Retail Project\Python section


In [7]:
import os
os.getcwd()
os.chdir(r"C:\Users\Amir\Dropbox\Workshop demo\Retail Project\Datasets\to be cleaned")

#### Unzip and Load CSV

In [ ]:
import zipfile
zip_ref = zipfile.ZipFile('orders.csv.zip')
zip_ref.extractall()
zip_ref.close()

In [9]:
df = pd.read_csv(r"C:\Users\Amir\Dropbox\Workshop demo\Retail Project\Datasets\to be cleaned\orders.csv", na_values= ['Not Available', 'unknown'])

In [10]:
df.head()

,Order Id,Order Date,Ship Mode,Segment,Country,City,State,Postal Code,Region,Category,Sub Category,Product Id,cost price,List Price,Quantity,Discount Percent
0,1,01/03/2023,Second Class,Consumer,United States,Henderson,Kentucky,42420,South,Furniture,Bookcases,FUR-BO-10001798,240,260,2,2.0
1,2,15/08/2023,Second Class,Consumer,United States,Henderson,Kentucky,42420,South,Furniture,Chairs,FUR-CH-10000454,600,730,3,3.0
2,3,10/01/2023,Second Class,Corporate,United States,Los Angeles,California,90036,West,Office Supplies,Labels,OFF-LA-10000240,10,10,2,5.0
3,4,18/06/2022,Standard Class,Consumer,United States,Fort Lauderdale,Florida,33311,South,Furniture,Tables,FUR-TA-10000577,780,960,5,2.0
4,5,13/07/2022,Standard Class,Consumer,United States,Fort Lauderdale,Florida,33311,South,Office Supplies,Storage,OFF-ST-10000760,20,20,2,5.0


In [11]:
print(len(df.columns))
print(df.columns)

16
Index(['Order Id', 'Order Date', 'Ship Mode', 'Segment', 'Country', 'City',
       'State', 'Postal Code', 'Region', 'Category', 'Sub Category',
       'Product Id', 'cost price', 'List Price', 'Quantity',
       'Discount Percent'],
      dtype='object')


In [56]:
df.dtypes

Order Id              int64
Order Date           object
Ship Mode            object
Segment              object
Country              object
City                 object
State                object
Postal Code           int64
Region               object
Category             object
Sub Category         object
Product Id           object
cost price            int64
List Price            int64
Quantity              int64
Discount Percent    float64
dtype: object

object dtype means it's categorical type.

#### Perform Cleaning
- Categorical Attributes
- Numerical Attributes

In [57]:
df.isnull().sum()

Order Id             0
Order Date           0
Ship Mode           13
Segment              0
Country              0
City                 0
State                0
Postal Code          0
Region               0
Category             0
Sub Category         0
Product Id           0
cost price           0
List Price           0
Quantity             0
Discount Percent    11
dtype: int64

In [58]:
# We've divided our main dataframe into two dataframes, numerical & categorical.
# So, it will help us to deal with missing values in a specific manner.
df_numeric = df.select_dtypes(include=['number'])
df_categorical = df.select_dtypes(include=['object'])

#### Deal with the Categorical Attribute Missing Values

In [59]:
df_categorical.isnull().sum()

Order Date       0
Ship Mode       13
Segment          0
Country          0
City             0
State            0
Region           0
Category         0
Sub Category     0
Product Id       0
dtype: int64

In [60]:
df_categorical['Ship Mode'].unique()

array(['Second Class', 'Standard Class', nan, 'First Class', 'Same Day'],
      dtype=object)

In [61]:
# the most common value of the attribute
df_categorical['Ship Mode'].mode()

0    Standard Class
Name: Ship Mode, dtype: object

In [62]:
# Replace 'nan' values with 'Standard Class' which is present at index 0
df_categorical['Ship Mode'].fillna(df_categorical['Ship Mode'].mode())


0         Second Class
1         Second Class
2         Second Class
3       Standard Class
4       Standard Class
             ...      
9989      Second Class
9990    Standard Class
9991    Standard Class
9992    Standard Class
9993      Second Class
Name: Ship Mode, Length: 9994, dtype: object

In [63]:
df_categorical.isnull().sum()

Order Date       0
Ship Mode       13
Segment          0
Country          0
City             0
State            0
Region           0
Category         0
Sub Category     0
Product Id       0
dtype: int64

We notice that even if the code was running, we still have missing values not replaced. That is basically, because we are dealing with series and fillna() function cannot fill with a list at each column. So, we have to specify the index number in order to replace the missing values with 'Standard Class' which is present at index 0. 

In [64]:
df_categorical['Ship Mode']=df_categorical['Ship Mode'].fillna(df_categorical['Ship Mode'].mode().iloc[0])


In [65]:
df_categorical.isnull().sum()

Order Date      0
Ship Mode       0
Segment         0
Country         0
City            0
State           0
Region          0
Category        0
Sub Category    0
Product Id      0
dtype: int64

#### Deal with the Numerical Attribute Missing Values
To deal with Numeric dtype, we use mean() function whereas in a categorical attribute we use mode() function.

In [66]:
df_numeric['Discount Percent']=df_numeric['Discount Percent'].fillna(df_numeric['Discount Percent'].mean())

In [67]:
df_numeric.isnull().sum()

Order Id            0
Postal Code         0
cost price          0
List Price          0
Quantity            0
Discount Percent    0
dtype: int64

#### Let's merge two dataframes
We need to specify either we are merging two data sets in a row /column fashion order(0,1).
- Column-wise(Side by Side): In this case, i want to merge them in a column fashion order (side by side). For that, axis=1. The number of rows should ideally be the same or else we get NanS.
- Row-wise(Top-Bottom stacking): axis=0, stacks the second df below the first (row-wise). The number and names of columns should ideally match otherwise we et NaNs in mismatched columns.


In [68]:
final_df = pd.concat([df_numeric,df_categorical], axis=1)
final_df.isnull().sum()

Order Id            0
Postal Code         0
cost price          0
List Price          0
Quantity            0
Discount Percent    0
Order Date          0
Ship Mode           0
Segment             0
Country             0
City                0
State               0
Region              0
Category            0
Sub Category        0
Product Id          0
dtype: int64

#### Feature Engineering

In [69]:
final_df = final_df.rename (columns={'Order Id': 'order_id', 'City': 'city'})
final_df.columns = final_df.columns.str.lower().str.replace(' ', '_')

1. Create columns 'Discount Price', 'Final Price', 'Profit'
2. Drop columns 'Discount Percent', 'List Price', 'Cost Price'

In [70]:
final_df['discount'] = final_df['list_price']* final_df['discount_percent']*.01
final_df['sale_price']= final_df['list_price'] - final_df['discount']
final_df['profit'] = final_df['sale_price'] - final_df['cost_price']
final_df

,order_id,postal_code,cost_price,list_price,quantity,discount_percent,order_date,ship_mode,segment,country,city,state,region,category,sub_category,product_id,discount,sale_price,profit
0,1,42420,240,260,2,2.0,01/03/2023,Second Class,Consumer,United States,Henderson,Kentucky,South,Furniture,Bookcases,FUR-BO-10001798,5.2,254.8,14.8
1,2,42420,600,730,3,3.0,15/08/2023,Second Class,Consumer,United States,Henderson,Kentucky,South,Furniture,Chairs,FUR-CH-10000454,21.9,708.1,108.1
2,3,90036,10,10,2,5.0,10/01/2023,Second Class,Corporate,United States,Los Angeles,California,West,Office Supplies,Labels,OFF-LA-10000240,0.5,9.5,-0.5
3,4,33311,780,960,5,2.0,18/06/2022,Standard Class,Consumer,United States,Fort Lauderdale,Florida,South,Furniture,Tables,FUR-TA-10000577,19.2,940.8,160.8
4,5,33311,20,20,2,5.0,13/07/2022,Standard Class,Consumer,United States,Fort Lauderdale,Florida,South,Office Supplies,Storage,OFF-ST-10000760,1.0,19.0,-1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9989,9990,33180,30,30,3,4.0,18/02/2023,Second Class,Consumer,United States,Miami,Florida,South,Furniture,Furnishings,FUR-FU-10001889,1.2,28.8,-1.2
9990,9991,92627,70,90,2,4.0,17/03/2023,Standard Class,Consumer,United States,Costa Mesa,California,West,Furniture,Furnishings,FUR-FU-10000747,3.6,86.4,16.4
9991,9992,92627,220,260,2,2.0,07/08/2022,Standard Class,Consumer,United States,Costa Mesa,California,West,Technology,Phones,TEC-PH-10003645,5.2,254.8,34.8
9992,9993,92627,30,30,4,3.0,19/11/2022,Standard Class,Consumer,United States,Costa Mesa,California,West,Office Supplies,Paper,OFF-PA-10004041,0.9,29.1,-0.9


In [71]:
# drop columns not needed anymore from our data set
final_df.drop(columns=['list_price', 'discount_percent'], errors='ignore', inplace=True)

In [84]:
final_df['order_date'] = pd.to_datetime(final_df['order_date'],format="%Y-%m-%d")

In [85]:
print(final_df)

      order_id  postal_code  cost_price  quantity order_date       ship_mode  \
0            1        42420         240         2 2023-03-01    Second Class   
1            2        42420         600         3 2023-08-15    Second Class   
2            3        90036          10         2 2023-01-10    Second Class   
3            4        33311         780         5 2022-06-18  Standard Class   
4            5        33311          20         2 2022-07-13  Standard Class   
...        ...          ...         ...       ...        ...             ...   
9989      9990        33180          30         3 2023-02-18    Second Class   
9990      9991        92627          70         2 2023-03-17  Standard Class   
9991      9992        92627         220         2 2022-08-07  Standard Class   
9992      9993        92627          30         4 2022-11-19  Standard Class   
9993      9994        92683         210         2 2022-07-17    Second Class   

        segment        country         

#### Store cleaned CSV

In [86]:
final_df.to_csv('data.csv')
